# Final Project – Introduction to Machine Learning (Group 32)
## Loan Approval Prediction – Binary Classification
**Tamar & Tala – Spring 2025**

This notebook walks through our end-to-end machine learning project:
- Data exploration 
- Feature processing 
- Model training 
- Evaluation 
- Final prediction submission 

## 1. Load Libraries and Data
_We’ll start by loading the data and setting up useful libraries._

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, classification_report

import missingno as msno


from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PowerTransformer



# Load the data
train = pd.read_csv('train.csv', na_values=["?"])
test = pd.read_csv('test.csv', na_values=["?"])
train.head()

,customer_id,Age,Employment_Type,A,Education_Level,Years_of_Education,Marital_Status,Job_Type,Household_Role,Ethnicity,...,Investment_Gain,Investment_Loss,Weekly_Work_Hours,Country_of_Residence,C,Preferred_Communication_Method,D,B,E,Loan_Approval
0,115892,90.0,NaN,77053,HS-grad,9.0,Widowed,NaN,Not-in-family,White,...,0.0,4356.0,40,United-States,104.906221,Phone_Call,2.865629,NaN,170.887465,0
1,115893,82.0,Private,132870,HS-grad,9.0,Widowed,Exec-managerial,Not-in-family,White,...,0.0,4356.0,18,United-States,96.358501,mail,5.528583,NaN,156.630201,0
2,115895,54.0,Private,140359,7th-8th,4.0,Divorced,Machine-op-inspct,Unmarried,White,...,0.0,3900.0,40,United-States,115.529631,mail,3.816915,NaN,165.635557,0
3,115896,41.0,Private,264663,Some-college,10.0,Separated,Prof-specialty,Own-child,White,...,0.0,3900.0,40,United-States,85.732506,NaN,5.416363,NaN,123.379007,0
4,115897,34.0,Private,216864,HS-grad,9.0,Divorced,Other-service,Unmarried,White,...,NaN,3770.0,45,United-States,115.218443,Mail,6.453932,NaN,155.262131,0


## 2. Exploratory Data Analysis (EDA)
_Let's understand the data structure, spot missing values, and explore correlations._


**Answer in markdown:**
- Are there outliers?
- Are there missing values?
- Any early ideas on important features?

In [7]:
n_rows,n_columns = train.shape
(n_rows,n_columns)

(27676, 21)

In [ ]:
train.describe()

In [ ]:
train.info()

Column B is completely empty (100% missing), so we will remove it from the dataset.
Several columns, such as Employment_Type, Job_Type and Ethnicity have a high percentage of missing values.
We'll need to decide whether to fill them or drop them.

_Visualizing the variables to check their distributions:_

In [ ]:
# Histograms and count plots of each explanatory variable

for col in train.columns:
    plt.figure(figsize=(15, 4))
    
    # For numeric variables
    if pd.api.types.is_numeric_dtype(train[col]):
        sns.histplot(train[col].dropna(), kde=True)
        plt.title(f"Distribution of {col}")
    
    # For categorical variables 
    else:
        sns.countplot(x=col, data=train, order=train[col].value_counts().index)
        plt.xticks(rotation=45)
        plt.title(f"Countplot of {col}")
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Bar plots of approval rates for categorical variables

categorical_vars = [
    'Employment_Type', 'Education_Level', 'Marital_Status', 'Job_Type',
    'Household_Role', 'Ethnicity', 'Gender', 'Country_of_Residence',
    'Preferred_Communication_Method'
]

for col in categorical_vars:
    plt.figure(figsize=(15, 4))
    sns.barplot(x=col, y='Loan_Approval', data=train, estimator='mean', order=train[col].value_counts().index)
    plt.xticks(rotation=45)
    plt.title(f'Approval Rate by {col}')
    plt.ylabel('Approval Rate (Mean of Loan_approval)')
    plt.tight_layout()
    plt.show()

In [ ]:
#Box plots of approved vs. rejected for numerical variables
train['Loan_Approval_str'] = train['Loan_Approval'].map({0: 'Rejected', 1: 'Approved'})

# Remove columns that shouldn't be plotted
numeric_cols = [col for col in train.select_dtypes(include='number').columns if col not in ['customer_id', 'Loan_Approval']]

# Create one boxplot per feature vs Loan Approval
for col in numeric_cols:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x='Loan_Approval_str', y=col, data=train)
    plt.title(f'{col} by Loan Approval')
    plt.xlabel('Loan Approval')
    plt.ylabel(col)
    plt.tight_layout()
    plt.show()

**Key Insights from Feature Distributions**
1. Employment_Type, Country_of_Residence and Ethnicity are highly imbalanced.
To reduce sparsity and improve generalization, we will group the less frequent categories into broader, more meaningful groups.
2. A is right-skewed, so a log transformation may help normalize its distribution.
3. Investment_Gain and Investment_Loss are extremely skewed with large outliers, so a log transformation might help, and merging them into a single Net_Investment feature may provide a clearer signal.
4. Preferred_Communication_Method has inconsistent values due to variations in capitalization and wording (e.g., "Email" vs "email"), so we unified similar values.
5. Based on the approval rate bar plot for Household_Role, we observed clear patterns across the categories.
We grouped them into three broader categories with similar approval behavior to simplify the feature and improve model stability.
6. Column B is completely empty, so we decided to remove it.
7.  We simplified the Country_of_Residence feature by grouping it into three categories: USA, Mexico, and Other. The vast majority of people were from the United States, and the approval rate among other countries wasn't very different.
8.  We simplified the Employment_Type feature by grouping it into four categories: Private, not private and, and non-working.  The vast majority of people were private, and the noticeable difference in the approval rate was between non-private to non-working.



In [ ]:
#Creating a heatmap

# Select only numeric columns
numeric_df = train.select_dtypes(include='number')

# Compute the correlation matrix
correlation_matrix = numeric_df.corr()

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", square=True)
plt.title("Correlation Heatmap")
plt.tight_layout()
plt.show()

In [ ]:
# Checking which variables are most correlated with Loan_approval (only linear) - this is the Loan_Approval row in the heatmap
numeric_df = train.select_dtypes(include='number')
correlations = numeric_df.corr()['Loan_Approval'].sort_values(ascending=False)
print(correlations)

**Key Insights from the heat map**
C and E are highly correlated (r = 0.87), indicating redundancy.
Neither has a strong linear correlation with the target, but E shows a slightly stronger signal (r = 0.125). Even though E has more missing values (as we will see later on), it’s still more useful, so we’ll keep E and drop C. 

We checked for overlap between Employment_Type and Job_Type using a cross-tabulation heatmap.
The results show no strong or consistent relationship between the two — most employment types are associated with a wide variety of job types.
Therefore, we concluded that the two features capture distinct information and chose to keep both.

In [ ]:
crosstab = pd.crosstab(train['Employment_Type'], train['Job_Type'], normalize='index')
plt.figure(figsize=(14, 7))
sns.heatmap(crosstab, cmap="Blues", annot=True, fmt=".2f")
plt.title("Employment Type vs Job Type")
plt.show()

# Changes based on the key insights:

In [8]:
# 1. Removing columns B and C
train = train.drop(['B', 'C'], axis=1)
test = test.drop(['B', 'C'], axis=1)
n_columns -= 2

In [14]:
# Handle missing Job_Type with knn
num_cols = [ 'Years_of_Education']          
cat_cols = ['Employment_Type', 'Household_Role']          
knn_col = 'Job_Type'  

# 1) Encode all categoricals (including the target column) into ordinal codes:
#    We'll impute missing codes later, so give them a placeholder value.
job_type_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

# 3) Build a ColumnTransformer that
#    - one-hot encodes all categoricals
#    - scales numerics
#    - then runs the KNN imputer on *all* columns
preprocessor = ColumnTransformer([
    ('encode_cat', job_type_encoder, cat_cols + [knn_col]),
    ('scale_num', StandardScaler(), num_cols)
])


'''
# 4) Validate properly:
scores = cross_val_score(
    job_type_pipe, train.loc[:, train.columns != 'Loan_Approval'], train['Loan_Approval'],
    cv=5, scoring='accuracy'
)
print("KNN-impute CV accuracy:", scores.mean())
'''

'\n# 4) Validate properly:\nscores = cross_val_score(\n    job_type_pipe, train.loc[:, train.columns != \'Loan_Approval\'], train[\'Loan_Approval\'],\n    cv=5, scoring=\'accuracy\'\n)\nprint("KNN-impute CV accuracy:", scores.mean())\n'

In [15]:
# Handle missing Employment_Type with knn
num_cols = ['Age', 'Years_of_Education', 'Weekly_Work_Hours']          
cat_cols = ['Job_Type', 'Marital_Status']          
knn_col = 'Employment_Type'  

# 1) Encode all categoricals (including the target column) into ordinal codes:
#    We'll impute missing codes later, so give them a placeholder value.
emp_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

# 3) Build a ColumnTransformer that
#    - one-hot encodes all categoricals
#    - scales numerics
#    - then runs the KNN imputer on *all* columns
preprocessor = ColumnTransformer([
    ('encode_cat', emp_encoder, cat_cols + [knn_col]),
    ('scale_num', StandardScaler(), num_cols)
])


'''
# 4) Validate properly:
scores = cross_val_score(
    emp_type_pipe, train.loc[:, train.columns != 'Loan_Approval'], train['Loan_Approval'],
    cv=5, scoring='accuracy'
)
print("KNN-impute CV accuracy:", scores.mean())
'''

'\n# 4) Validate properly:\nscores = cross_val_score(\n    emp_type_pipe, train.loc[:, train.columns != \'Loan_Approval\'], train[\'Loan_Approval\'],\n    cv=5, scoring=\'accuracy\'\n)\nprint("KNN-impute CV accuracy:", scores.mean())\n'

In [16]:
# ------------ Grouping functions ------------

# Grouping Preferred_Communication_Method
def group_pref_com(pref):
    if pref == "Mail" or pref == "mail":
        return "Mail"
    if pref == "Email" or pref == "email":
        return "EMail"
    if pref == "Phone" or pref == "phone" or pref == "phone call":
        return "Phone"
    else:
        return "Other"

# Grouping Country_of_Residence
def group_country(country):
    if country == "United-States":
        return "USA"
    elif country == "Mexico":
        return "Mexico"
    else:
        return "Other"
    
# Grouping Education_Level
def group_education(edu):
    pre_school = ['Preschool']
    school = ['1st-4th', '5th-6th', '9th', '10th', '11th', '12th', '7th-8th']
    post_hs = ['HS-grad', 'Some-college']
    assoc = ['Assoc-voc', 'Assoc-acdm']
    higher = ['Masters', 'Bachelors']
    postgraduate = ['Doctorate', 'Prof-school']
    
    if edu in pre_school:
        return 'pre_school'
    elif edu in school:
        return 'school'
    elif edu in post_hs:
        return 'post_hs'
    elif edu in assoc:
        return 'assoc'
    elif edu in higher:
        return 'higher'
    elif edu in postgraduate:
        return 'postgraduate'
    else:
        return 'Other'
    
# Grouping Household_role
def group_household_role(role):
    if role in ['Husband', 'Wife']:
        return 'Spouse'
    else:
        return 'Other'

# Grouping Employment_Type
def group_employment(emp):
    if emp == 'Private':
        return 'Private'
    elif emp in ['Self-emp-not-inc', 'Self-emp-inc', 'Local-gov', 'State-gov', 'Federal-gov']:
        return 'Not-Private'
    elif emp in ['Without-pay', 'Never-worked']:
        return 'Non-working'
    else:
        return 'Other'
    
# Grouping Ethnicity:
def group_ethnicity(eth):
    if eth == 'White':
        return 'White'
    elif eth == 'Black':
        return 'Black'
    else:
        return 'Other'
    
# 1. Mapping dictionary for Years_of_Education
edu_to_years = {
    'Preschool'    : 0,
    '1st-4th'      : (1 + 2 + 3 + 4) / 4,
    '5th-6th'      : 5.5,
    '7th-8th'      : 7.5,
    '9th'          : 9,
    '10th'         : 10,
    '11th'         : 11,
    '12th'         : 12,
    'HS-grad'      : 12,
    'Some-college' : (12 + 13 + 14 + 15) / 4,
    'Assoc-voc'    : 14,
    'Assoc-acdm'   : 14,
    'Bachelors'    : 16,
    'Masters'      : 18,
    'Prof-school'  : 19,
    'Doctorate'    : 20
}


def group_edu_years(X):
    #X is a 2D array: [Years_of_Education, Education_Level
    
    # Convert columns to panda series to be able to map
    years_of_education = pd.Series(X[:, 0])
    education_level = pd.Series(X[:, 1])
    
    # Map Education_Level → numeric years
    mapped_years = education_level.map(edu_to_years)
    
    # Fill NaN in the original 'years' with the mapped values
    filled = years_of_education.fillna(mapped_years)
    
    # Return as a single-column DataFrame
    return filled.to_frame(name='Years_of_Education')

In [17]:
from sklearn.preprocessing import FunctionTransformer

# ------------ function transformers ------------

pref_com_group = FunctionTransformer(
    func=lambda X: pd.Series(X.ravel()).map(group_pref_com).to_frame(),
    validate=False
)

country_group = FunctionTransformer(
    func=lambda X: pd.Series(X.ravel()).map(group_country).to_frame(),
    validate=False
)

edu_level_group = FunctionTransformer(
    func=lambda X: pd.Series(X.ravel()).map(group_education).to_frame(),
    validate=False
)

house_hold_group = FunctionTransformer(
    func=lambda X: pd.Series(X.ravel()).map(group_household_role).to_frame(),
    validate=False
)

emp_type_group = FunctionTransformer(
    func=lambda X: pd.Series(X.ravel()).map(group_employment).to_frame(),
    validate=False
)

ethnicity_group = FunctionTransformer(
    func=lambda X: pd.Series(X.ravel()).map(group_ethnicity).to_frame(),
    validate=False
)

edu_years_group = FunctionTransformer(
    func=group_edu_years,
    validate=False
)

In [28]:
# ------------ pipelines ------------

knn_imputer = KNNImputer(n_neighbors=3, weights='uniform')

# Preferred_Communication_Method pipeline
pref_pipeline = Pipeline([
    ('group', pref_com_group),  
    ('impute', SimpleImputer(strategy='constant', fill_value='Missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Country_of_Residence pipeline
country_pipeline = Pipeline([
    ('group', country_group),
    ('impute', SimpleImputer(strategy='most_frequent')),  # or 'constant' if you prefer
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Education_Level pipeline
edu_pipeline = Pipeline([
    ('group', edu_level_group),
    ('impute', SimpleImputer(strategy='constant', fill_value='Missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Employment_Type pipeline
employment_pipeline = Pipeline([
    ('group', emp_type_group),
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Ethnicity pipeline
ethnicity_pipeline = Pipeline([
    ('group', ethnicity_group),
    ('impute', SimpleImputer(strategy='constant', fill_value='Missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Household pipeline
Household_pipeline = Pipeline([
    ('group', house_hold_group),
    ('impute', SimpleImputer(strategy='constant', fill_value='Missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Years of education pipeline
edu_years_pipeline = Pipeline([
    ('fill_years', edu_years_group),
    ('impute_years', SimpleImputer(strategy='mean'))
])

# job type of education pipeline
job_type_pipe = Pipeline([
    ('preproc', preprocessor),
    ('impute_knn', knn_imputer),
    ('clf', RandomForestClassifier())
])

# Employment type pipeline
emp_type_pipe = Pipeline([
    ('preproc', preprocessor),
    ('impute_knn', knn_imputer),
    ('clf', RandomForestClassifier())
])

# columns without special adjustments:
categorical_col = ['Gender', 'Marital_Status']
numerical_col = ['E']
skw_numerical_col = ['Age', 'Investment_Gain', 'Investment_Loss']

# Pipeline for categorical features
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='Missing')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Pipeline for numerical features (mean imputation)
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])

# Pipeline for numerical features (median imputation) - for skewed columns
skw_numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])


# Combine pipelines into a ColumnTransformer
preprocessor = ColumnTransformer([
    ('pref',       pref_pipeline,         ['Preferred_Communication_Method']),
    ('country',    country_pipeline,      ['Country_of_Residence']),
    ('edu_level',  edu_pipeline,          ['Education_Level']),
    ('edu_years',  edu_years_pipeline,    ['Years_of_Education', 'Education_Level']),
    ('employment', employment_pipeline,    ['Employment_Type']),
    ('ethnicity',  ethnicity_pipeline,    ['Ethnicity']),
    ('household',  Household_pipeline,    ['Household_Role']),
    ('cat_basic',  cat_pipeline,          ['Gender', 'Marital_Status']),                        # Other categorical columns
    ('num_basic',  num_pipeline,          ['E']),                                               # Simple numeric column
    ('skw_num',    skw_numerical_pipeline,['Age', 'Investment_Gain', 'Investment_Loss'])        # Skewed numeric columns
])



In [44]:
import numpy as np
X = train.drop(columns=['Loan_Approval'])
X_transformed = preprocessor.fit_transform(X)


# 3. Check for missing values
print("Total missing values after preprocessing:", np.isnan(X_transformed).sum())

# Optional: break if something is wrong
assert not np.isnan(X_transformed).any(), "❌ Still missing values after preprocessing!"


Total missing values after preprocessing: 0


In [34]:
X_transformed

array([[1.000e+00, 0.000e+00, 0.000e+00, ..., 9.000e+01, 0.000e+00,
        4.356e+03],
       [1.000e+00, 0.000e+00, 0.000e+00, ..., 8.200e+01, 0.000e+00,
        4.356e+03],
       [1.000e+00, 0.000e+00, 1.000e+00, ..., 5.400e+01, 0.000e+00,
        3.900e+03],
       ...,
       [1.000e+00, 0.000e+00, 0.000e+00, ..., 2.700e+01, 0.000e+00,
        0.000e+00],
       [1.000e+00, 0.000e+00, 0.000e+00, ..., 5.800e+01, 0.000e+00,
        0.000e+00],
       [0.000e+00, 1.000e+00, 0.000e+00, ..., 2.200e+01, 0.000e+00,
        0.000e+00]])

In [45]:
train

,customer_id,Age,Employment_Type,A,Education_Level,Years_of_Education,Marital_Status,Job_Type,Household_Role,Ethnicity,Gender,Investment_Gain,Investment_Loss,Weekly_Work_Hours,Country_of_Residence,Preferred_Communication_Method,D,E,Loan_Approval
0,115892,90.0,NaN,77053,HS-grad,9.0,Widowed,NaN,Not-in-family,White,Female,0.0,4356.0,40,United-States,Phone_Call,2.865629,170.887465,0
1,115893,82.0,Private,132870,HS-grad,9.0,Widowed,Exec-managerial,Not-in-family,White,Female,0.0,4356.0,18,United-States,mail,5.528583,156.630201,0
2,115895,54.0,Private,140359,7th-8th,4.0,Divorced,Machine-op-inspct,Unmarried,White,Female,0.0,3900.0,40,United-States,mail,3.816915,165.635557,0
3,115896,41.0,Private,264663,Some-college,10.0,Separated,Prof-specialty,Own-child,White,Female,0.0,3900.0,40,United-States,NaN,5.416363,123.379007,0
4,115897,34.0,Private,216864,HS-grad,9.0,Divorced,Other-service,Unmarried,White,Female,NaN,3770.0,45,United-States,Mail,6.453932,155.262131,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27671,148447,53.0,Private,321865,Masters,14.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,NaN,0.0,40,United-States,mail,6.719535,151.618445,1
27672,148448,22.0,Private,310152,Some-college,10.0,Never-married,Protective-serv,Not-in-family,White,Male,0.0,0.0,40,United-States,Mail,3.334034,134.985353,0
27673,148449,27.0,Private,257302,Assoc-acdm,12.0,Married-civ-spouse,Tech-support,Wife,White,Female,NaN,0.0,38,United-States,Email,6.204021,148.281633,0
27674,148451,58.0,Private,151910,HS-grad,9.0,Widowed,Adm-clerical,Unmarried,NaN,Female,0.0,0.0,40,United-States,Email,2.986238,NaN,0


In [40]:
# Understanding missing values - Check how many missing values there are in each column and if it has a pattern
missing = X_transformed_df.isnull().sum()
missing_percent = (missing / len(train)) * 100

print(missing_percent.sort_values(ascending=False))

# Plot missing value matrix
#msno.matrix(train)

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
dtype: float64


In [43]:
train.head(5)

,customer_id,Age,Employment_Type,A,Education_Level,Years_of_Education,Marital_Status,Job_Type,Household_Role,Ethnicity,Gender,Investment_Gain,Investment_Loss,Weekly_Work_Hours,Country_of_Residence,Preferred_Communication_Method,D,E,Loan_Approval
0,115892,90.0,NaN,77053,HS-grad,9.0,Widowed,NaN,Not-in-family,White,Female,0.0,4356.0,40,United-States,Phone_Call,2.865629,170.887465,0
1,115893,82.0,Private,132870,HS-grad,9.0,Widowed,Exec-managerial,Not-in-family,White,Female,0.0,4356.0,18,United-States,mail,5.528583,156.630201,0
2,115895,54.0,Private,140359,7th-8th,4.0,Divorced,Machine-op-inspct,Unmarried,White,Female,0.0,3900.0,40,United-States,mail,3.816915,165.635557,0
3,115896,41.0,Private,264663,Some-college,10.0,Separated,Prof-specialty,Own-child,White,Female,0.0,3900.0,40,United-States,NaN,5.416363,123.379007,0
4,115897,34.0,Private,216864,HS-grad,9.0,Divorced,Other-service,Unmarried,White,Female,NaN,3770.0,45,United-States,Mail,6.453932,155.262131,0


In [ ]:

#Doing a log transformation while still distinguishing losses vs. gains.
train['Net_Investment'] = train['Investment_Gain'] - train['Investment_Loss']
test['Net_Investment'] = test['Investment_Gain'] - test['Investment_Loss']

train['Log_Net_Investment'] = np.sign(train['Net_Investment']) * np.log2(abs(train['Net_Investment']) + 1)
test['Log_Net_Investment'] = np.sign(test['Net_Investment']) * np.log2(abs(test['Net_Investment']) + 1)


# Side-by-side boxplots 
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
sns.boxplot(x=train['Net_Investment'], ax=axes[0])
axes[0].set_title('Original Net_Investment')
axes[0].set_xlabel('Net_Investment')

sns.boxplot(x=train['Log_Net_Investment'], ax=axes[1])
axes[1].set_title('Log-Transformed Net_Investment')
axes[1].set_xlabel('Log_Net_Investment')

plt.tight_layout()
plt.show()

# Side-by-side histograms
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
sns.histplot(train['Net_Investment'].dropna(), kde=True, ax=axes[0])
axes[0].set_title('Distribution of Net_Investment')
axes[0].set_xlabel('Net_Investment')

sns.histplot(train['Log_Net_Investment'].dropna(), kde=True, ax=axes[1])
axes[1].set_title('Distribution of Log-Transformed Net_Investment')
axes[1].set_xlabel('Log_Net_Investment')

plt.tight_layout()
plt.show()


In [ ]:


train['log_gain'] = np.log2(train['Investment_Gain'] + 1)
train['log_loss'] = np.log2(train['Investment_Loss'] + 1)
train['net_investment_log'] = train['log_gain'] - train['log_loss']

plt.hist(train['net_investment_log'])
plt.show()

sns.histplot(train['net_investment_log'].dropna(), kde=True)
plt.title(f"Distribution of net_investment_log")



We now have a few ways of dealing with the investments: 
1. keeping each one in the original way.
2. keeping each one with a log transformation.
3. Log on each one and then difference
4. Difference and then log on each one.

we will need to do some cross-validation and check which one gives the best results.

We applied a log transformation to Net_Investment to reduce skew and compress extreme outliers.
The resulting distribution is more symmetric, so we will keep it.


We applied a log transformation to column A to reduce skew and potentially improve its relationship with the target.
However, the correlation with Loan_Approval remained low, so we won't use it.

In [ ]:
pd.crosstab(train['Loan_Approval'], train['Gender'])
pd.crosstab(train['Loan_Approval'], train['Ethnicity'], normalize='index')
pd.crosstab(train['Loan_Approval'], train['Education_Level'], normalize='index')
pd.crosstab(train['Loan_Approval_str'], train['Gender'], normalize='index')

# Doesn't really help because of the unbalanced proportions. Add an explanation.


In [ ]:
# AFTER CHANGING 

# Checking which variables are most correlated with Loan_approval (only linear):
numeric_df = train.select_dtypes(include='number')
correlations = numeric_df.corr()['Loan_Approval'].sort_values(ascending=False)
print(correlations)

## 3. Data Preprocessing
_Handle missing values, encode categorical features, normalize, etc._

**✏️ Answer in markdown:**
- Did you normalize the data? Why?
- How did you handle categorical variables?
- Did you apply PCA or feature selection?
- Did you drop or create new features?


In [ ]:
######## TRY AGAIN AFTER ADDING AGE
'''
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

df = train  

# 1. Define features & split known vs missing
features = ['Weekly_Work_Hours', 'Years_of_Education', 'Age']
known   = df[df['Investment_Gain'].notnull() ].copy()
missing = df[df['Investment_Gain'].isnull()].copy()

X_known = known[features]
y_known = known['Investment_Gain']

# 2. Hold out 20% for validation
X_tr, X_val, y_tr, y_val = train_test_split(
    X_known, y_known,
    test_size=0.2,
    random_state=42
)

# 3. Train on the 80%
lr = LinearRegression()
lr.fit(X_tr, y_tr)

# 4. Predict on the hold‐out and evaluate
y_pred = lr.predict(X_val)
print(f"R² on hold-out: {r2_score(y_val, y_pred):.3f}")
print(f"MAE on hold-out: {mean_absolute_error(y_val, y_pred):.3f}")

# 5. (Optional) Retrain on ALL known data and impute missings
lr_full = LinearRegression()
lr_full.fit(X_known, y_known)

X_missing = missing[features]
df.loc[df['Investment_Gain'].isnull(), 'Investment_Gain'] = lr_full.predict(X_missing)
'''


We tried to handle the missing values with linear regression, but R^2 was 0.02, so we just impute with the median.

The plot shows that column B is completely empty, so we should remove it. The rest of the columns have some missing values, but there doesn’t seem to be any clear pattern.

In [ ]:
# 1. Create a boolean mask for missingness - job type
mask_Job_Type = train['Job_Type'].isna()            # True where missing
train['MissingFlag_Job_Type'] = mask_Job_Type.map({True: 'Missing', False: 'Present'})

# 2. Summary statistics for numeric vars
numeric_cols = ['Log_Net_Investment']
#numeric_cols = ['Age', 'Years_of_Education', 'Weekly_Work_Hours', 'Net_Investment', 'A', 'D', 'E']
print(train.groupby('MissingFlag_Job_Type')[numeric_cols].describe().unstack(1))

''' bring back after - just sick of the big graphs.
# 4. Frequencies for other categorical vars
cat_cols = ['Marital_Status', 'Household_Role', ]
for col in cat_cols:
    freq = pd.crosstab(train['MissingFlag_Job_Type'], train[col], normalize='index')
    print(f'\nRelative frequency of {col}:\n', freq)
    freq.plot.bar(stacked=True)
    plt.title(f'{col} by missingness')
    plt.ylabel('Proportion')
    plt.show()

'''

In [ ]:
# 1. Create a boolean mask for missingness with Investment_Gain to see if we can delete it to use YJ
mask_Investment_Gain = train['Investment_Gain'].isna()            # True where missing
train['MissingFlag_Investment_Gain'] = mask_Investment_Gain.map({True: 'Missing', False: 'Present'})

# 2. Summary statistics for numeric vars
numeric_cols = ['E']
#numeric_cols = ['Age', 'Years_of_Education', 'Weekly_Work_Hours', 'A', 'D', 'E']
print(train.groupby('MissingFlag_Investment_Gain')[numeric_cols].describe().unstack(1))


''' bring back after - just sick of the big graphs.

# 4. Frequencies for other categorical vars
cat_cols = ['Marital_Status', 'Household_Role', ]
for col in cat_cols:
    freq = pd.crosstab(train['MissingFlag_Investment_Gain'], train[col], normalize='index')
    print(f'\nRelative frequency of {col}:\n', freq)
    freq.plot.bar(stacked=True)
    plt.title(f'{col} by missingness')
    plt.ylabel('Proportion')
    plt.show()
'''

In [ ]:
# 1. Create a boolean mask for missingness with Ethnicity
mask_Ethnicity = train['Ethnicity'].isna()            # True where missing
train['MissingFlag_Ethnicity'] = mask_Ethnicity.map({True: 'Missing', False: 'Present'})

# 2. Summary statistics for numeric vars
numeric_cols = ['D']
#numeric_cols = ['Age', 'Years_of_Education', 'Weekly_Work_Hours', 'A', 'D', 'E']
print(train.groupby('MissingFlag_Ethnicity')[numeric_cols].describe().unstack(1))

'''
#bring back after - just sick of the big graphs.

# 4. Frequencies for other categorical vars
cat_cols = ['Marital_Status', 'Household_Role', 'Employment_Type', 'Job_Type', 'Gender', 'Country_of_Residence', 'Preferred_Communication_Method']
for col in cat_cols:
    freq = pd.crosstab(train['MissingFlag_Ethnicity'], train[col], normalize='index')
    print(f'\nRelative frequency of {col}:\n', freq)
    freq.plot.bar(stacked=True)
    plt.title(f'{col} by missingness')
    plt.ylabel('Proportion')
    plt.show()
'''

In [ ]:
# 1. Create a boolean mask for missingness with Ethnicity
mask_Ethnicity = train['Preferred_Communication_Method'].isna()            # True where missing
train['MissingFlag_Ethnicity'] = mask_Ethnicity.map({True: 'Missing', False: 'Present'})

# 2. Summary statistics for numeric vars
numeric_cols = ['Weekly_Work_Hours']
#numeric_cols = ['Age', 'Years_of_Education', 'Weekly_Work_Hours', 'A', 'D', 'E']
print(train.groupby('MissingFlag_Ethnicity')[numeric_cols].describe().unstack(1))

'''
#bring back after - just sick of the big graphs.

# 4. Frequencies for other categorical vars
cat_cols = ['Marital_Status', 'Household_Role', 'Employment_Type', 'Job_Type', 'Gender', 'Country_of_Residence', 'Preferred_Communication_Method']
for col in cat_cols:
    freq = pd.crosstab(train['MissingFlag_Ethnicity'], train[col], normalize='index')
    print(f'\nRelative frequency of {col}:\n', freq)
    freq.plot.bar(stacked=True)
    plt.title(f'{col} by missingness')
    plt.ylabel('Proportion')
    plt.show()
'''

In [ ]:
# 1. Create a boolean mask for Job tyoe with Ethnicity
mask_Ethnicity = train['Job_Type'].isna()            # True where missing
train['MissingFlag_Ethnicity'] = mask_Ethnicity.map({True: 'Missing', False: 'Present'})

# 2. Summary statistics for numeric vars
numeric_cols = ['Weekly_Work_Hours']
#numeric_cols = ['Age', 'Years_of_Education', 'Weekly_Work_Hours', 'A', 'D', 'E']
print(train.groupby('MissingFlag_Ethnicity')[numeric_cols].describe().unstack(1))

'''
#bring back after - just sick of the big graphs.

# 4. Frequencies for other categorical vars
cat_cols = ['Marital_Status', 'Household_Role', 'Employment_Type', 'Job_Type', 'Gender', 'Country_of_Residence', 'Preferred_Communication_Method']
for col in cat_cols:
    freq = pd.crosstab(train['MissingFlag_Ethnicity'], train[col], normalize='index')
    print(f'\nRelative frequency of {col}:\n', freq)
    freq.plot.bar(stacked=True)
    plt.title(f'{col} by missingness')
    plt.ylabel('Proportion')
    plt.show()
'''

In [ ]:
# 1. Create a boolean mask for missingness with Ethnicity
mask_Ethnicity = train['Employment_Type'].isna()            # True where missing
train['MissingFlag_Ethnicity'] = mask_Ethnicity.map({True: 'Missing', False: 'Present'})

# 2. Summary statistics for numeric vars
numeric_cols = ['Weekly_Work_Hours']
#numeric_cols = ['Age', 'Years_of_Education', 'Weekly_Work_Hours', 'A', 'D', 'E']
print(train.groupby('MissingFlag_Ethnicity')[numeric_cols].describe().unstack(1))

'''
#bring back after - just sick of the big graphs.

# 4. Frequencies for other categorical vars
cat_cols = ['Marital_Status', 'Household_Role', 'Employment_Type', 'Job_Type', 'Gender', 'Country_of_Residence', 'Preferred_Communication_Method']
for col in cat_cols:
    freq = pd.crosstab(train['MissingFlag_Ethnicity'], train[col], normalize='index')
    print(f'\nRelative frequency of {col}:\n', freq)
    freq.plot.bar(stacked=True)
    plt.title(f'{col} by missingness')
    plt.ylabel('Proportion')
    plt.show()
'''

i see that the missing and present have a really different distribution. so i used knn

We flagged missing Investment_Gain and saw identical “Missing” vs. “Present” distributions, so it’s MCAR. We will drop those rows to be able to apply Yeo–Johnson to the rest.- DONT THINK SO

In [ ]:
from scipy.stats import ttest_ind

a = train.loc[train['MissingFlag_Job_Type']=='Missing', 'Weekly_Work_Hours'].dropna()
b = train.loc[train['MissingFlag_Job_Type']=='Present','Weekly_Work_Hours'].dropna()
t_stat, p_val = ttest_ind(a, b, equal_var=False)
print(f'p-value = {p_val}')


#notes:
C is handeled with mean bacause its distributed normally.

for values with more than 5 percent of missing data we checked if the data is missed randomly or not.
we can see with the t-test that the missing job_type are not random. so we will use a more sofisticated way to deal with the missing values

## 4. Model Training – Basic Models
_Start simple. Train Logistic Regression or KNN._

In [ ]:
from sklearn.linear_model import LogisticRegression

# This transforms (missing data and stuff like that) AND does the classifier as well.


model = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', LogisticRegression())
])

model.fit(train[[categorical_col]], y_train)

## 5. Model Training – Advanced Models
Choose 3 models from:
- Decision Tree
- Random Forest or AdaBoost
- Support Vector Machine
- Multi-Layer Perceptron (Neural Net)

_Include basic hyperparameter tuning or explain defaults._

## 6. Evaluation
- Build confusion matrix for at least one model
- Generate ROC curves (using K-Fold CV)
- Report AUC

**✏️ Answer in markdown:**
- Does any model overfit?
- Which model generalizes best?
- Final choice justification

## 7. Final Prediction on Test Set
_Predict probabilities (not binary labels!) and create submission file._

In [ ]:
# Example:
# final_model.predict_proba(test)[:, 1]  # ← probability of 'approved'
# submission = pd.DataFrame({
#     'customer_id': test['customer_id'],
#     'loan_approval': predictions
# })
# submission.to_csv('Submission_group_32.csv', index=False)

## 8. Appendix / Extra Notes
_Use this area for extra analysis, failed attempts, or ideas you explored._